# Phase 2: Story Structure

This phase is the most important on this project. We are making the story structure so we basically have 3 steps for this:

* Start defining the ASP approach.
* Use the Sentiment Analysis to create emotional integration
* The system generates multiple valid outlines that satisfy our constrains, ensuring both logical coherence and emotional resonance.

So in more detail what we will be doing in code is defining multiple functions, here are the key components:

1. Scene structure foundation.
2. Narrative Functions.
3. Constrain System.
4. Implementation features.
 

In [3]:
# import the principal libraries
import pandas as pd 
import numpy as np 
# import torch
import os
from groq import Groq, Client
from dotenv import load_dotenv 
load_dotenv()

import dspy
import tqdm as notebook_tqdm
# import typos
from typing import Dict, Tuple, Optional, List, Set, Optional

# import other libraries
from enum import Enum 
from dataclasses import dataclass

In [5]:
open_ai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
# import keys from .env
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
client = Client(
    api_key=groq_api_key
    )
llmo = dspy.LM('openai/gpt-4o-mini', api_key=open_ai_api_key)
dspy.configure(lm=llmo)
# Load the clean document
# doc_path = "cleaned_text.txt"
# with open(doc_path, "r") as f:
#     INPUT_FILE = f.read()

In [47]:
test_file = """The Eagle, the Hare, and the Beetle
A hare was being chased by an eagle, and seeing herself lost, she begged a beetle for help, pleading for its assistance.
The beetle asked the eagle to spare his friend. But the eagle, despising the insignificance of the beetle, devoured the hare in his presence.
From then on, seeking revenge, the beetle observed the places where the eagle laid its eggs, and rolling them, knocked them to the ground. Seeing herself driven away from wherever she went, the eagle turned to Zeus asking for a safe place to lay her eggs.
Zeus offered to let her place them in his lap, but the beetle, seeing this escape tactic, made a small ball of dung, flew up and dropped it on Zeus's lap.
Zeus then stood up to shake off that filth, and unknowingly threw the eggs to the ground. That is why since then, eagles do not lay eggs during the season when beetles come out to fly.
Moral: Never despise what seems insignificant, for there is no being so weak that it cannot reach you."""

### 1. Structure Scene Fundation

In [48]:
class ExtractRelevantInformation(dspy.Signature):
    """Extract the relevant information and key themes about this story"""
    input_text: str = dspy.InputField()
    reasoning: str = dspy.OutputField(desc="Step by step reasoning and extraction of key themes")
    moral_of_the_story: str = dspy.OutputField(desc="The moral lesson or teaching (Enseñanza) of the story")
    key_themes: str = dspy.OutputField(desc="Key themes and main characters identified in the story")

class StoryCreation(dspy.Signature):
    """Based on the key themes and moral of the story, create an entertaining story"""
    input_text: str = dspy.InputField()
    themes: str = dspy.InputField()
    moral: str = dspy.InputField()
    story: str = dspy.OutputField(desc="Create an entertaining story incorporating the themes and moral")


In [49]:
class StoryStructure(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extract_relevant_information = dspy.ChainOfThought(ExtractRelevantInformation)
        self.story_creation = dspy.ChainOfThought(StoryCreation)

    def forward(self, input_text):
        # First, extract themes and moral from the initial text
        extracted_info = self.extract_relevant_information(
            input_text=input_text
        )
        
        # Then create the story using the extracted information
        story = self.story_creation(
            input_text=input_text,
            themes=extracted_info.key_themes,
            moral=extracted_info.moral_of_the_story
        )
        
        return {
            'themes': extracted_info.key_themes,
            'moral': extracted_info.moral_of_the_story,
            'story': story.story,
            'reasoning': extracted_info.reasoning
        }

In [50]:
# Initialize the story generator
generator = StoryStructure()

# Generate the story and return all components
result = generator(test_file)

In [51]:
result["story"]


'Once upon a time in a lush green valley, there lived a swift hare named Hazel. She was known for her speed and agility, but one fateful day, while frolicking in the meadow, she caught the eye of a fierce eagle named Edgar. With his sharp talons and keen eyesight, Edgar swooped down, determined to make Hazel his next meal.\n\nIn a panic, Hazel darted through the tall grass, her heart racing. Just as she thought she was cornered, she spotted a small beetle named Benny, who was busy rolling a tiny ball of dung. "Please, Benny! Help me! The eagle is after me!" she cried, her voice trembling.\n\nBenny looked up at the towering eagle, who was circling above with a menacing glare. "Edgar, please spare my friend!" he pleaded, his voice barely a whisper against the wind. But Edgar, with a disdainful chuckle, replied, "What can a tiny beetle do? She is nothing but a meal to me!" With that, he swooped down and snatched Hazel away, leaving Benny in shock.\n\nHeartbroken and furious, Benny vowed t

In [52]:
def print_story_result(result_dict):
    print("\n" + "="*50 + " TEMAS " + "="*50)
    print(result_dict['themes'])
    
    print("\n" + "="*50 + " MORAL " + "="*50)
    print(result_dict['moral'])
    
    print("\n" + "="*50 + " HISTORIA " + "="*50)
    print(result_dict['story'])
    
    print("\n" + "="*50 + " RAZONAMIENTO " + "="*50)
    print(result_dict['reasoning'])
    print("\n" + "="*120)


In [53]:
# Print the result
print_story_result(result)


================================================== TEMAS ==================================================
Key themes include vulnerability, the quest for help, revenge, the underestimation of the weak, and the consequences of one's actions. Main characters are the hare, the eagle, and the beetle.

================================================== MORAL ==================================================
Never despise what seems insignificant, for there is no being so weak that it cannot reach you.

================================================== HISTORIA ==================================================
Once upon a time in a lush green valley, there lived a swift hare named Hazel. She was known for her speed and agility, but one fateful day, while frolicking in the meadow, she caught the eye of a fierce eagle named Edgar. With his sharp talons and keen eyesight, Edgar swooped down, determined to make Hazel his next meal.

In a panic, Hazel darted through the tall grass, her hear

In [54]:
with open("new_story.txt", 'w', encoding='utf-8') as f:
        f.write(result["story"])

In [55]:
import torch
from transformers import AutoTokenizer
import soundfile as sf

from parler_tts import ParlerTTSForConditionalGeneration


In [69]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")


/home/mau/miniconda3/envs/conda_podcastllama_env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id":

In [70]:

prompt = result["story"]
description = "A female speaker delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."


In [71]:

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)


Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors


In [72]:

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()


In [74]:
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)